# Exercise: Heating Values of Gaseous Fuels

This notebook contains a helper function `heating_value(fuel)` that returns
both the LHV and HHV of a specified gaseous fuel mixture using Cantera.

Use it to:
- evaluate the heating values of different fuels (e.g. `'CH4'`, `'H2'`,
  `'CO'`, or mixtures such as `'CH4:0.8, H2:0.2'`),
- compare HHV and LHV and relate the difference to the amount of water
  produced during combustion,
- explore how the heating value per kg of fuel depends on fuel composition.

After reading the code, add a few cells where you call `heating_value` for
several fuels and summarise your findings in a short table or plot.


In [1]:
import cantera as ct
import numpy as np

gas = ct.Solution("gri30.yaml")
water = ct.Water()
# Set liquid water state, with vapor fraction x = 0
water.TQ = 298, 0
h_liquid = water.h
# Set gaseous water state, with vapor fraction x = 1
water.TQ = 298, 1
h_gas = water.h


def heating_value(fuel):
    """Returns the LHV and HHV for the specified fuel"""
    gas.TP = 298, ct.one_atm
    gas.set_equivalence_ratio(1.0, fuel, "O2:1.0")
    h1 = gas.enthalpy_mass
    Y_fuel = gas[fuel].Y[0]

    # complete combustion products
    X_products = {
        "CO2": gas.elemental_mole_fraction("C"),
        "H2O": 0.5 * gas.elemental_mole_fraction("H"),
        "N2": 0.5 * gas.elemental_mole_fraction("N"),
    }

    gas.TPX = None, None, X_products
    Y_H2O = gas["H2O"].Y[0]
    h2 = gas.enthalpy_mass
    LHV = -(h2 - h1) / Y_fuel / 1e6
    HHV = -(h2 - h1 + (h_liquid - h_gas) * Y_H2O) / Y_fuel / 1e6
    return LHV, HHV


fuels = ["H2", "CH4", "C2H6", "C3H8", "NH3", "CH3OH"]
print("fuel   LHV (MJ/kg)   HHV (MJ/kg)")
for fuel in fuels:
    LHV, HHV = heating_value(fuel)
    print(f"{fuel:8s} {LHV:7.3f}      {HHV:7.3f}")

fuel   LHV (MJ/kg)   HHV (MJ/kg)
H2       119.952      141.780
CH4       50.025       55.511
C2H6      47.511       51.901
C3H8      46.352       50.343
NH3       18.604       22.479
CH3OH     21.104       23.851
